# Softmax Classification

In [3]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [4]:
#For reproducibility
torch.manual_seed(1)

### softmax를 활용하여 숫자를 확률로 변환하는 과정

In [5]:
z = torch.FloatTensor([1,2,3])

pytorch has a softmax function.

In [6]:
hypothesis = F.softmax(z, dim = 0)
print(hypothesis)

tensor([0.0900, 0.2447, 0.6652])


In [7]:
hypothesis.sum()

tensor(1.)

# Cross Entropy Loss

 multi-class classification을 위해서는 Cross Entropy Loss를 써야함

In [9]:
z = torch.rand(3,5,requires_grad = True)
hypothesis = F.softmax(z, dim = 1)
print(hypothesis)

tensor([[0.1664, 0.1871, 0.1737, 0.2695, 0.2033],
        [0.2002, 0.1783, 0.2218, 0.1944, 0.2054],
        [0.1809, 0.2380, 0.2318, 0.1084, 0.2409]], grad_fn=<SoftmaxBackward>)


In [10]:
y = torch.randint(5,(3,)).long()
print(y)

tensor([2, 0, 3])


In [12]:
y_one_hot = torch.zeros_like(hypothesis)
y_one_hot.scatter_(1,y.unsqueeze(1),1)

tensor([[0., 0., 0., 1., 0.],
        [0., 0., 1., 0., 0.],
        [0., 0., 0., 1., 0.]])

In [14]:
cost = (y_one_hot * -torch.log(hypothesis)).sum(dim = 1).mean()
print(cost)

tensor(1.6800, grad_fn=<MeanBackward0>)


### Cross-entropy Loss with torch.nn.functional

In [16]:
# Low Lsvel
torch.log(F.softmax(z,dim = 1))

tensor([[-1.7935, -1.6760, -1.7504, -1.3114, -1.5929],
        [-1.6086, -1.7244, -1.5062, -1.6381, -1.5826],
        [-1.7096, -1.4354, -1.4617, -2.2223, -1.4236]], grad_fn=<LogBackward>)

In [17]:
# High Level
F.log_softmax(z, dim = 1)

tensor([[-1.7935, -1.6760, -1.7504, -1.3114, -1.5929],
        [-1.6086, -1.7244, -1.5062, -1.6381, -1.5826],
        [-1.7096, -1.4354, -1.4617, -2.2223, -1.4236]],
       grad_fn=<LogSoftmaxBackward>)

In [18]:
# Low Lsvel
(y_one_hot * -torch.log(F.softmax(z, dim=1))).sum(dim=1).mean()

tensor(1.6800, grad_fn=<MeanBackward0>)

In [19]:
# High Level
F.nll_loss(F.log_softmax(z, dim=1), y)

tensor(1.6800, grad_fn=<NllLossBackward>)

pytorch also has F.cross_entropy that combines F.log_softmax() and f.nll_loss()

In [21]:
F.cross_entropy(z,y)

tensor(1.6800, grad_fn=<NllLossBackward>)

## Training with Low-level Cross Entropy Loss

In [12]:
x_train = [[1,2,1,1],
           [2,1,3,2],
           [3,1,3,4],
           [4,1,5,5],
           [1,7,5,5],
           [1,2,5,6],
           [1,6,6,6],
           [1,7,7,7]]
y_train = [2, 2, 2, 1, 1, 1, 0, 0]
x_train = torch.FloatTensor(x_train)
y_train = torch.LongTensor(y_train)

In [13]:
#모델 초기화
W = torch.zeros((4,3),requires_grad = True)
b = torch.zeros(1, requires_grad = True)

#optimizer 설정
optimizer = optim.SGD([W,b], lr = 0.1)

nb_epochs = 1000
for epoch in range(nb_epochs + 1):
    
    #cost 계산(1)
    #hypothesis = F.softmax(x_train.matmul(W) + b, dim = 1)
    #y_one_hot = torch.zeros_like(hypothesis)
    #y_one_hot.scatter_(1, y_train.unsqueeze(1),1) #오류 원인 찾기
    #cost = (y_one_hot * -torch.log(F.softmax(hypothesis,dim=1))).sum(dim=1) 
    
    #cost 계산(2)
    z = x_train.matmul(W) + b
    cost = F.cross_entropy(z, y_train)
    
    #cost로 H(x) 개선
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()
    
    #100번 마다 로그 출력
    if epoch % 100 == 0:
        print('Epoch {:4d} / {} Cost: {:.6f}'.format(epoch, nb_epochs, cost.item()))

Epoch    0 / 1000 Cost: 1.098612
Epoch  100 / 1000 Cost: 0.761050
Epoch  200 / 1000 Cost: 0.689991
Epoch  300 / 1000 Cost: 0.643229
Epoch  400 / 1000 Cost: 0.604117
Epoch  500 / 1000 Cost: 0.568255
Epoch  600 / 1000 Cost: 0.533922
Epoch  700 / 1000 Cost: 0.500291
Epoch  800 / 1000 Cost: 0.466908
Epoch  900 / 1000 Cost: 0.433507
Epoch 1000 / 1000 Cost: 0.399962


In [14]:
class SoftmaxClassifierModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear = nn.Linear(4,3)
        
    def forward(self, x):
        return self.linear(x)

In [17]:
model = SoftmaxClassifierModel()

In [18]:
#optimizer 설정
optimizer = optim.SGD(model.parameters(), lr = 0.1)

nb_epochs = 1000
for epoch in range(nb_epochs + 1):
    
    #H(x) 계산
    prediction = model(x_train)
    
    # cost 계산
    cost = F.cross_entropy(prediction, y_train)
    
    #cost로 H(x) 개선
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()
    
    #100번 마다 로그 출력
    if epoch % 20 == 0:
        print('Epoch {:4d} / {} Cost: {:.6f}'.format(epoch, nb_epochs, cost.item()))

Epoch    0 / 1000 Cost: 2.288633
Epoch   20 / 1000 Cost: 0.943391
Epoch   40 / 1000 Cost: 0.801895
Epoch   60 / 1000 Cost: 0.750842
Epoch   80 / 1000 Cost: 0.719368
Epoch  100 / 1000 Cost: 0.695717
Epoch  120 / 1000 Cost: 0.676300
Epoch  140 / 1000 Cost: 0.659513
Epoch  160 / 1000 Cost: 0.644499
Epoch  180 / 1000 Cost: 0.630745
Epoch  200 / 1000 Cost: 0.617923
Epoch  220 / 1000 Cost: 0.605808
Epoch  240 / 1000 Cost: 0.594244
Epoch  260 / 1000 Cost: 0.583114
Epoch  280 / 1000 Cost: 0.572334
Epoch  300 / 1000 Cost: 0.561838
Epoch  320 / 1000 Cost: 0.551575
Epoch  340 / 1000 Cost: 0.541505
Epoch  360 / 1000 Cost: 0.531597
Epoch  380 / 1000 Cost: 0.521824
Epoch  400 / 1000 Cost: 0.512164
Epoch  420 / 1000 Cost: 0.502601
Epoch  440 / 1000 Cost: 0.493120
Epoch  460 / 1000 Cost: 0.483707
Epoch  480 / 1000 Cost: 0.474352
Epoch  500 / 1000 Cost: 0.465047
Epoch  520 / 1000 Cost: 0.455782
Epoch  540 / 1000 Cost: 0.446552
Epoch  560 / 1000 Cost: 0.437349
Epoch  580 / 1000 Cost: 0.428169
Epoch  600